In [133]:
"""

Chapter 5: Exploring Compilers. 
Exploring Numba and PyPy.
_______________________________

Numba exposes specialized types using the signatures attribute. Right after the sum_sq definition.

If we call this function with a specific argument Numba compiles a specialized version

An individual signature can be passed as a tuple containg type we would like to accept.
If we want to specify an array of a specific type, use slicing operator [:] on type itself

Another way to declare signatures is through type strings. Also pass multiple signatue

Numba provides a function, called inspect_types,
helps understand how effictive type inference was and which operations were optimized.

_______________________________

Numba and Numpy:
    Originally developed to increase performance of code using NP arr.
    Currently, many NP features are implemented efficiaently by compiler

Universal functions: np broadcast function:
    U funcs can be dif in np by the scalar version and the
    np.verctorize func to enhance the func with broadcasting feature

"""


'\n\nChapter 5: Exploring Compilers. \nExploring Numba and PyPy.\n_______________________________\n\nNumba exposes specialized types using the signatures attribute. Right after the sum_sq definition.\n\nIf we call this function with a specific argument Numba compiles a specialized version\n\nAn individual signature can be passed as a tuple containg type we would like to accept.\nIf we want to specify an array of a specific type, use slicing operator [:] on type itself\n\nAnother way to declare signatures is through type strings. Also pass multiple signatue\n\nNumba provides a function, called inspect_types,\nhelps understand how effictive type inference was and which operations were optimized.\n\n_______________________________\n\nNumba and Numpy:\n    Originally developed to increase performance of code using NP arr.\n    Currently, many NP features are implemented efficiaently by compiler\n\nUniversal functions: np broadcast function:\n    U funcs can be dif in np by the scalar versi

In [134]:
import numba as nb


def sum_sq(a):
    resultofSum = 0
    N  = len(a)
    for i in range(N):
        resultofSum += a[i]
    return resultofSum



In [135]:
# first steps with numba

@nb.jit

def sum_sq(a):
    resultofSum = 0
    N  = len(a)
    for i in range(N):
        resultofSum += a[i]
    return resultofSum

import numpy as np 

x = np.random.rand(10000)

# Original

%timeit sum_sq.py_func(x)

# numba

%timeit sum_sq(x)



1.43 ms ± 63.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


C:\Users\JBarr\anaconda3\envs\dev\lib\site-packages\IPython\core\magics\execution.py:165: RuntimeWarning: coroutine 'fetch_urls' was never awaited
  it = itertools.repeat(None, number)


11 µs ± 86.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [136]:

sum_sq.signatures

x = np.random.rand(1000).astype('float64')

sum_sq(x)

display(sum_sq.signatures)

xx = np.random.rand(1000).astype('float32')
sum_sq(xx)

display(sum_sq.signatures)

[(array(float64, 1d, C),)]

C:\Users\JBarr\AppData\Roaming\Python\Python37\site-packages\numba\core\ir.py:194: RuntimeWarning: coroutine 'fetch_urls' was never awaited
  return type(self)(self.filename, line, col)


[(array(float64, 1d, C),), (array(float32, 1d, C),)]

In [137]:
#@nb.jit("float64(float64[:])")
@nb.jit(["float64(float64[:])",
        "float64(float32[:])"])
def sum_sq(a):
    resultofSum = 0
    N  = len(a)
    for i in range(N):
        resultofSum += a[i]
    return resultofSum



In [138]:
sum_sq(x.astype("float32"))

498.23671908999677

In [139]:
sum_sq.inspect_types()

sum_sq (array(float64, 1d, A),)
--------------------------------------------------------------------------------
# File: C:\Users\JBarr\AppData\Local\Temp/ipykernel_15272/4038124245.py
# --- LINE 2 --- 

@nb.jit(["float64(float64[:])",

        # --- LINE 3 --- 

        "float64(float32[:])"])

# --- LINE 4 --- 

def sum_sq(a):

    # --- LINE 5 --- 
    # label 0
    #   a = arg(0, name=a)  :: array(float64, 1d, A)
    #   resultofSum = const(int, 0)  :: Literal[int](0)
    #   resultofSum.2 = resultofSum  :: float64
    #   del resultofSum

    resultofSum = 0

    # --- LINE 6 --- 
    #   $6load_global.1 = global(len: <built-in function len>)  :: Function(<built-in function len>)
    #   N = call $6load_global.1(a, func=$6load_global.1, args=[Var(a, 4038124245.py:5)], kws=(), vararg=None, target=None)  :: (array(float64, 1d, A),) -> int64
    #   del $6load_global.1

    N  = len(a)

    # --- LINE 7 --- 
    #   jump 14
    # label 14
    #   $16load_global.0 = global(range: <cla

In [140]:
def contor_py(a, b):
    return int(0.5 * (a + b) * ( a + b + 1 ) + b)

In [141]:
np.vectorize
def contor(a, b):
    return int(0.5 * (a + b) * ( a + b + 1 ) + b)
    

In [142]:
# Py
%timeit contor_py(10, 33)

329 ns ± 13.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [143]:
# Numba

%timeit contor(10, 33)

314 ns ± 9.24 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [144]:
# Numpy

%timeit (0.5 * (10.0 + 33.3) * ( 10.0 + 33.3 + 1.0 ) + 33)

6.72 ns ± 0.129 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [145]:
# In Python we define a Node class as follows:
# Wecanmanage the collection of Nodeinstances creating another class,
# called LinkedList. The class keeps track of head of list


class Node:
    def __init__(self, value):
        self.next = None
        self.value=  value

class  LinkedList:
    def __init__(self):
        self.head = None
    def push_front(self, value):
        if self.head == None:
            self.head = Node(value)
        else:
            # We replace the head
            new_head = Node(value)           
            new_head.next = self.head
            self.head = new_head
        # For debugging implement LinkedList.show that print each element
    def show(self):
        node = self.head
        while node is not None:
            print(node.value)
            node = node.next

In [146]:
first = LinkedList()
first.push_front(1)
first.push_front(2)
first.push_front(3)
first.show()


3
2
1


In [147]:
@nb.jit
def sum_list(first):
    res = 0
    node = first.head
    while node is not None:
        
        result += node.value
        node = node.next
    return result
    

In [148]:
javone = LinkedList()
[javone.push_front(i) for i in range(10)]


[None, None, None, None, None, None, None, None, None, None]

In [149]:
node_type = nb.deferred_type()

node_spec = [
    ('next', nb.optional(node_type)),
    ('value', nb.int64)
]


In [150]:
node_spec

[('next', OptionalType(DeferredType#1732800261896)), ('value', int64)]

In [151]:

@nb.jitclass(node_spec)

class Node:
    def __init__(self, value):
        self.next = None
        self.value=  value


node_type.define(Node.class_type.instance_type)

AttributeError: module 'numba' has no attribute 'jitclass'

In [ ]:

"""
Chapter 6 Implementing Concurrency: Asychoronous programming is a programming paradigm that deal
with slow and unpredictable resources. 



Callbacks: To makethe code start  working on other tasks  we need to find away to avoid blocking
the program flow so that the restof the program can work on other task.
"""




'\nChapter 6 Implementing Concurrency: Asychoronous programming is a programming paradigm that deal\nwith slow and unpredictable resources. \n\n\n\nCallbacks: To makethe code start  working on other tasks  we need to find away to avoid blocking\nthe program flow so that the restof the program can work on other task.\n'

In [ ]:
import time

def network_request(number):
    time.sleep(1.0)
    return {"success": True, "result": number **  2}

def fetch_square(number):
    REPOS = network_request(number)
    if REPOS['success']:
        print("Result is: {}".format(REPOS['result']))


fetch_square(2)

fetch_square(3)

fetch_square(4)

Result is: 4
Result is: 9
Result is: 16


In [ ]:
# will block program execution then print()

def wait_and_print(msg):
    time.sleep(1.0)
    print(msg)

In [ ]:
# A callback is a functin the will be called when the timer expires.
# we also call the Timer.start method  to activate timer.

import threading

def wait_and_print_async(msg):
    def callback():
        print(msg)

    timer = threading.Timer(1.0, callback)
    timer.start()

In [ ]:
%%time
wait_and_print('Hello world');


Hello world
Wall time: 1.01 s


In [ ]:

%%time
wait_and_print_async('Hello world')

Wall time: 997 µs
Hello world


In [ ]:
def network_request_async(number, on_done):

    def  timer_done():
        on_done({"success": True, "result": number ** 2})
        timer = threading.Timer(1.0, timer_done)
        timer.start()

def on_done(result):
    print(result)

network_request_async(2, on_done)

In [ ]:
network_request_async(2, on_done)
network_request_async(3, on_done)
network_request_async(4, on_done)
print("After submmision")

After submmision


In [ ]:
def fetch_square(number):
    def on_done(response):
        if response['success']:
            print("Result is: {}".format(response['result']))

    network_request_async(number, on_done)

In [ ]:
fetch_square(4)


Hello world


In [ ]:
"""
Futures are a more conbenient pattern used to track results of asynchronous calls.
of the requested resources and that we are waiting to become available. 

To attact a callback it is sufficient to pass a function to the Future.add_done_callback method.
When the task completes, the func will be called with Fut instance as first arg
result can is retrived suing Future.result() method

In practical apps futures would be handled by frameworks
_________________________________

In many asynchronous frameworks, the coordination of concurrent tasks is managed by an event loop.
With E.L we continuosly monitor status of various resources (network connections and databse queries) 
triggeating execution of callbacks when events take place (when resouce is ready of timer expires)

Events over Threading
every unit of execution never runs at the same time helping with shared resources.

"""

'\nFutures are a more conbenient pattern used to track results of asynchronous calls.\nof the requested resources and that we are waiting to become available. \n\nTo attact a callback it is sufficient to pass a function to the Future.add_done_callback method.\nWhen the task completes, the func will be called with Fut instance as first arg\nresult can is retrived suing Future.result() method\n\n'

In [ ]:
# Futures

import threading
import concurrent

fut = concurrent.futures.Future()

display(fut)

fut.set_result('Hello')

fut.result()



<Future at 0x1f184094cc8 state=pending>

'Hello'

In [ ]:

FuncFut = concurrent.futures.Future()


FuncFut.add_done_callback(lambda future: print(future.result(), flush=True))
display(FuncFut)

FuncFut.set_result('World')


<Future at 0x1f183feec88 state=pending>

World


In [ ]:
from concurrent.futures import Future

def network_request_async(number):
    future = Future()
    result = {"success": True, "result": number ** 2}
    timer = threading.Timer(1.0, lambda: future.set_result(result))
    timer.start()
    return future



fut =  network_request_async(2)


In [ ]:
from concurrent.futures import Future

def fet_square(number):
    futnum = network_request_async(number)

    def on_done_future(future):
        responce = future.result()
        if responce['success']:
            print("Result is: {}".format(responce["result"]))
    futnum.add_done_callback(on_done_future)

In [ ]:
# Event loops
import time

In [ ]:
class Timer:
    def __init__(self, timeout):
        self.timeout = timeout
        self.start = time.time()
    def done(self):
        return time.time() - self.start > self.timeout

    def on_timer_done(self, callback):
        self.callback = callback

# Determine is timer expired with loop that continuously checks status
# by bcalling Timer.done method

# implamenting timer this way, the flow of execution is never blocked and we can, work inside loop
# This method is often called lazy waiting

timer = Timer(1.0)

while True:
    if timer.done():
        print("Timer is done!")
        break


Timer is done!


In [ ]:
class Timer:
    def __init__(self, timeout):
        self.timeout = timeout
        self.start = time.time()
    def done(self):
        return time.time() - self.start > self.timeout

    def on_timer_done(self, callback):
        self.callback = callback

# Determine is timer expired with loop that continuously checks status
# by bcalling Timer.done method

# implamenting timer this way, the flow of execution is never blocked and we can, work inside loop
# This method is often called lazy waiting

timer = Timer(5.0)
timer.on_timer_done(lambda: print("Timer is done!"))

while True:
    if timer.done():
        timer.callback()
        break


Timer is done!


In [ ]:
"""
A nat way to implement mul timers is to add a few Timer instances to a list
and modidy E.L periodically check all the timers and dispatch the callbacks when requied.
"""

timers = []

timer1 = Timer(3.0)
timer1.on_timer_done(lambda: print("First Timer is done!"))

timer2 = Timer(2.0)
timer2.on_timer_done(lambda: print("Second Timer is done!"))

timers.append(timer1)

timers.append(timer2)

while True:
    for timer in timers:
        if timer.done():
            timer.callback()
            timers.remove(timer)
    if len(timers) == 0:
        break

Second Timer is done!
First Timer is done!


In [ ]:
"""
The Py standard lib include an event loop-based concurrency framework:The asyncio framework

Coroutines:
    A more nat wat to break up code execution into chunks. Allowing dev to code in synchronous syntax
    but will ececute assynchronouslly. Think of Corouties as function that can be stopped and resumed.

    You can think of a yield statement as a breakpoint where we stop and resume execution
        (while maintaining internal stateof genearator)
    Thid can be leveranged by event loop for concurrency.


    import asyncio

#loop = asyncio.get_event_loop()

#def callback():
 #   print("Hello, asyncio")
  #  loop.stop()



#loop.call_later(1.0, callback)
#loop.stop()

# To define coroutine with asyncio use async def statement

async def hello():
    print("Hello, Neato")

coro = hello()
loop = asyncio.get_event_loop()
loop.run_until_complete(coro)

loop.close()


async def wait_and_print(msg):
    await asyncio.sleep(2)
    print("Message: ", msg)
loop.run_until_complete(wait_and_print("Hello"))

SKIPPED REST OF SYINC EXAMPLES0
30

"""


def range_generator(n):
    i = 0
    while i < n:
        print('Generating value {}'.format(i))
        yield i
        i += 1
        
gen = range_generator(3)

display(gen)

display(next(gen))

display(next(gen))

display(next(gen))        

<generator object range_generator at 0x0000019370C1CD48>

Generating value 0


0

Generating value 1


1

Generating value 2


2

In [ ]:
# Inject values in the generator through the yield statement. 
# we need to issue a generator.send(None) before we start sending messages;
    # This is done to bootstarp function execution and bring us to first yield statement
import asyncio
def parrot():
    while True:
        message = yield
        print("Parrot says: {}".format(message))


gen = parrot()
display(gen.send(None))
display(gen.send("Hello"))
display(gen.send("World"))

None

Parrot says: Hello


None

Parrot says: World


None

In [ ]:
"""
Converting blocking code into non-blocking code:
An effective strategy for dealing with blocking code run it in seperate thread.
Threads are implemnted at OS level and allow parallel exectuin of blocking code.

The .submit method immediately schedules the function and returns a fut.
Manage execution of task in asyncio 
_____________________


"""


'\nConverting blocking code into non-blocking code:\nAn effective strategy for dealing with blocking code run it in seperate thread.\nThreads are implemnted at OS level and allow parallel exectuin of blocking code\n_____________________\n\n\n'

In [152]:
from concurrent.futures import ThreadPoolExecutor
import time 


executor = ThreadPoolExecutor(max_workers=6)

def wait_and_ret(msg):
    time.sleep(1)
    return msg


urls = ["https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD",
    "https://api.alternative.me/v2/ticker/Ethereum/?convert=USD",
    "https://api.alternative.me/v2/ticker/Chainlink/?convert=USD",
    "https://api.alternative.me/v2/ticker/Cosmos/?convert=USD",
    "https://api.alternative.me/v2/ticker/Algorand/?convert=USD"]

In [153]:
import os

import requests

import json

import msgpack

import pandas as pd

from dotenv import load_dotenv

import alpaca_trade_api as tradeapi

#from MCForecastTools import MCSimulation

import datetime

# Load the environment variables from the .env file
#by calling the load_dotenv function

load_dotenv()

pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)

async def fetch_urls(urls):
    respon = []
    for u in urls:
        respon.append(await loop.run_in_executor(executor, requests.get, u))
    
    return respon

loop.run_until_complete(fetch_urls(urls))

Python-dotenv could not parse statement starting at line 29
Python-dotenv could not parse statement starting at line 30


RuntimeError: This event loop is already running

In [170]:
def fetch_urls(urls):
    return asyncio.gather(*[loop.run_in_executor
    (executor, requests.get, u) for u in urls])

inN = fetch_urls(urls)


In [171]:

inN.result()
    

[<Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>]